## Run UDFs on Merlin live streams

If you want to use this with the simulated data source, run something like this in the background:

`libertem-live-mib-sim ~/Data/default.hdr --cached=MEM`

On Linux, `MEMFD` is also supported as a cache. Use `NONE` to deactivate the cache.

* Make sure to adjust the `SCAN_SIZE` below to match the scan of the data source!
* This notebook requires the `bqplot` extra of LiberTEM: `pip install libertem[bqplot]`

In [1]:
# Uncomment to use Matplotlib-based plots
# This requires ipympl and allows to capture Matplotlib plots as ipywidgets.
# %matplotlib widget

In [2]:
# set this to the host/port where the merlin data server is listening:
MERLIN_DATA_SOCKET = ('127.0.0.1', 6342)
MERLIN_CONTROL_SOCKET = ('127.0.0.1', 6341)
SCAN_SIZE = (128, 128)

In [3]:
import logging
import time

import numpy as np
import ipywidgets
from contextlib import contextmanager

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
# Sum all detector frames, result is a map of the detector
from libertem.udf.sum import SumUDF
# Sum up each detector frame, result is a bright field STEM image of the scan area
from libertem.udf.sumsigudf import SumSigUDF

# ImageGL-accelerated plot for fast live display
from libertem.viz.bqp import BQLive2DPlot
# Alternatively a version that uses the slower, but more mature Matplotlib
from libertem.viz.mpl import MPLLive2DPlot

INFO:empyre:Imported EMPyRe V-0.3.0 GIT-7531a074e8e81c3e02d65fad075edcd2c5408ad7


In [6]:
from libertem_live.api import LiveContext
from libertem_live.detectors.merlin import MerlinControl
from libertem_live.udf.monitor import SignalMonitorUDF

In [7]:
ctx = LiveContext()

### Camera setup routines

Different from offline processing, the shape, type and content of a dataset is not predetermined in live processing.
Instead, the data source has to be configured to supply the desired data. The functions here accept an acquisition object as a parameter to make it easier to configure a matching scan resolution.

In [8]:
def merlin_setup(c: MerlinControl, dwell_time=1e-3, depth=12, save_path=None):
    print("Setting Merlin acquisition parameters")
    # Here go commands to control the camera and the rest of the setup
    # to perform an acquisition.

    # The Merlin simulator currently accepts all kinds of commands
    # and doesn't respond like a real Merlin detector.
    c.set('CONTINUOUSRW', 1)
    c.set('ACQUISITIONTIME' , dwell_time * 1e3)  # Time in miliseconds
    c.set('COUNTERDEPTH', depth)
    c.set('TRIGGERSTART', 3)
    c.set('TRIGGERSTOP', 0)
    c.set('RUNHEADLESS', 1)

    if save_path is not None:
        c.set('FILEENABLE', 1)
        c.set('USETIMESTAMPING', 0)  # raw format with timestamping is buggy, we need to do it ourselves
        c.set('FILEFORMAT', 2)  # raw format, less overhead?
        c.set('FILEDIRECTORY', save_path)
    else:
        c.set('FILEENABLE', 0)


    print("Finished Merlin setup.")
        
def microscope_setup(dwell_time=1e-3):
    # Here go instructions to set dwell time and
    # other scan parameters
    # microscope.set_dwell_time(dwell_time)
    pass

def arm(c: MerlinControl):
    print("Arming Merlin...")
    c.cmd('STARTACQUISITION')
    print("Merlin ready for trigger.")
    

def set_nav(c: MerlinControl, aq):
    height, width = aq.shape.nav
    print("Setting resolution...")
    c.set('NUMFRAMESTOACQUIRE', height * width)
    c.set('NUMFRAMESPERTRIGGER', width)  # One trigger per scan line
    
    # microscope.configure_scan(shape=aq.shape.nav)

### Trigger function

A LiberTEM Live acquisition object should include a trigger callback function so that LiberTEM Live can set off the acquisition as soon as it has connected to the camera and is ready to receive data.

In [9]:
def trigger():
    print("Triggering!")
    # scan_engine.trigger_scan()

In [10]:
aq = ctx.prepare_acquisition(
    'merlin',
    trigger=trigger,
    scan_size=SCAN_SIZE,
    host=MERLIN_DATA_SOCKET[0],
    port=MERLIN_DATA_SOCKET[1],
    frames_per_partition=800,
    pool_size=2
)

In [11]:
udfs = [SumUDF(), SumSigUDF(), SignalMonitorUDF()]

In [12]:
LivePlot = BQLive2DPlot
# Uncomment below to use Matplotlib-based plotting
# See also the top of the notebook to select the correct matplotlib backend
# LivePlot = MPLLive2DPlot

p0 = LivePlot(aq, udfs[0])
p1 = LivePlot(aq, udfs[1])
p2 = LivePlot(aq, udfs[2])

In [13]:
outputs = []

for p in [p0, p1, p2]:
    # Capture the plots to display them in a grid later
    output = ipywidgets.Output()
    with output:
        p.display()
        # Some plot-specific tweaks for grid display
        if isinstance(p, BQLive2DPlot):
            p.figure.fig_margin={'top': 50, 'bottom': 0, 'left': 25, 'right': 25}
            p.figure.layout.width = '300px'
            p.figure.layout.height = '300px'
        elif isinstance(p, MPLLive2DPlot):
            p.fig.tight_layout()
            p.fig.set_size_inches((3, 3))
            p.fig.canvas.toolbar_position = 'bottom'
    outputs.append(output)

In [14]:
# Show the plot grid
ipywidgets.HBox(outputs)

### Sample output

The plots are not preserved when saving the notebook. They look like this:

![sample plot](run_on_merlin_data.png)

### Run one scan

The live plots above are updated with the results

In [15]:
c = MerlinControl(*MERLIN_CONTROL_SOCKET)
print("Connecting Merlin control...")
with c:
    merlin_setup(c)
    microscope_setup()

    set_nav(c, aq)
    arm(c)

ctx.run_udf(dataset=aq, udf=udfs, plots=[p0, p1, p2])
print("Finished.")

Connecting Merlin control...
Setting Merlin acquisition parameters
Finished Merlin setup.
Setting resolution...
Arming Merlin...
Merlin ready for trigger.
Triggering!
Finished.
